In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import functions as F


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Silver Data Exploration") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.default.parallelism", "12") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.sql.files.maxPartitionBytes", "128m") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.shuffle.service.enabled", "true") \
    .config("spark.locality.wait", "0s") \
    .getOrCreate()



In [6]:
btc = spark.read \
    .option("mergeSchema", "false") \
    .parquet("/app/data/bronze/bitcoin_data/") \
    .coalesce(2)

btc.show(20,truncate=False)

+-------------+-----------------+---------------------+---------------------+----+-----+---+
|timestamp    |price            |market_cap           |total_volume         |year|month|day|
+-------------+-----------------+---------------------+---------------------+----+-----+---+
|1740355537873|96221.55883939596|1.9083936974055684E12|1.563158446085855E10 |2025|2    |24 |
|1740355906059|96480.92770254097|1.9083936974055684E12|1.771748910737594E10 |2025|2    |24 |
|1740356207393|96458.6475277336 |1.9083936974055684E12|1.3561980045231249E10|2025|2    |24 |
|1740356438173|96356.52568488687|1.9110833469477192E12|1.6187084507414503E10|2025|2    |24 |
|1740356778287|96408.36100518158|1.9109748479960166E12|1.4687156118955452E10|2025|2    |24 |
|1740357097087|96390.21645628785|1.9123686486368752E12|1.6429408697221909E10|2025|2    |24 |
|1740357412101|96251.1123703611 |1.9095710802652363E12|1.6453226993530008E10|2025|2    |24 |
+-------------+-----------------+---------------------+---------------